In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,confusion_matrix,classification_report,accuracy_score
import copy

In [50]:
df=pd.read_csv("AdmissionDataset/data.csv")
threshold = 0.5

In [51]:
Y = df['Chance of Admit ']
Y = list(Y)
for i in range(len(Y)):
    if Y[i]>=threshold:
        Y[i]=1
    else:
        Y[i]=0
Y = pd.DataFrame({'Chance of Admit ':Y})
Y = Y['Chance of Admit ']

In [52]:
X = df.drop(['Chance of Admit ','Serial No.'],axis=1)
# Y = df['Chance of Admit ']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)
X_train_knn = copy.deepcopy(X_train)
Y_train_knn = copy.deepcopy(Y_train)
X_test_knn = copy.deepcopy(X_test)
Y_test_knn = copy.deepcopy(Y_test)

In [53]:
X_train = (X_train - X_train.mean())/X_train.std()
X_test = (X_test - X_test.mean())/X_test.std()

In [54]:
X_train = pd.concat([X_train,Y_train],axis=1)
ones = np.ones([X_train.shape[0],1])
Y_train = X_train.iloc[:,7:8].values
X_train = X_train.iloc[:,0:7]
X_train = np.concatenate((ones,X_train),axis=1)

In [55]:
learning_rate = 0.01
iterations = 1000
# theta = np.zeros(8) # 7 is the number of features
theta = np.zeros([1,8])
# theta.shape

In [56]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [57]:
def gradient_decent(X_train,Y_train,theta,learning_rate,iterations):
    
    for i in range(iterations):
        h = sigmoid(np.dot(X_train,theta.T))
        theta = theta - (learning_rate/len(X_train)) * np.sum(X_train * (h - Y_train), axis=0)
    
    return theta

In [58]:
g = gradient_decent(X_train,Y_train,theta,learning_rate,iterations)
g = g[0]

In [59]:
y_pred = []
for index,rows in X_test.iterrows():
    y = 0
    rows = list(rows)
    for i in range(len(rows)):
        y = y + rows[i]*g[i+1]
    y = y + g[0]
#     y_pred.append(y)
    y = sigmoid(y)
    if y >= threshold:
        y_pred.append(1)
    else:
        y_pred.append(0)

# Vectorized form
# ones = np.ones([X_test.shape[0],1])
# X_test = np.concatenate((X_test,ones),axis=1)
# y_pred = X_test @ g

In [60]:
(y_pred == Y_test).mean()
print(confusion_matrix(Y_test,y_pred))
# print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test, y_pred)*100)

[[ 0  5]
 [ 0 85]]
94.44444444444444


In [61]:
def euclidean_distance(x, y):   
    ans = np.sqrt(np.sum((x - y) ** 2))
    return ans

In [62]:
k=5

In [63]:
pred = []
for index,row in X_test_knn.iterrows():
    dist= []
    for index1,row1 in X_train_knn.iterrows():
        dist.append(euclidean_distance(row,row1))
        
    our = pd.DataFrame(
    {'a': dist,
     'b': Y_train_knn
    })
    our = our.sort_values(by=['a'])
    zero = 0
    one = 0
    i = 0
    for index,row in our.iterrows():
        if i >= k:
            break
        if row['b'] == 1:
            one = one + 1
        elif row['b'] == 0:
            zero = zero + 1 
        i = i + 1
    if one >= zero:
        pred.append(1)
    else:
        pred.append(0)

In [64]:
print(confusion_matrix(Y_test_knn,pred))
# print(classification_report(Y_test_knn,pred))
print(accuracy_score(Y_test_knn, pred)*100)

[[ 1  4]
 [ 1 84]]
94.44444444444444
